# Final Scoring in the Competition and Ranking!

__Submission__

![title](submission_score.png)

__Leaderboard as of 03/12/2022__

![title](leaderboard_1.png)

__Imports__

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

__Dataset__

In [2]:
digit = pd.read_csv("data/train.csv")
digit_test = pd.read_csv("data/test.csv")

__Training Dataset View__

In [3]:
digit.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


__Test Dataset View__

In [4]:
digit_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


__Get only values__

In [5]:
x = digit.drop('label', axis = 1).values
y = digit.label.values

In [6]:
x.shape

(42000, 784)

In [7]:
y.shape

(42000,)

__Re-scaling__

In [8]:
x = x/255.0
test_df = digit_test.values/255.0

__Reshaping__

In [9]:
x_train = x.reshape(x.shape[0], 28, 28,1)
test_df = test_df.reshape(digit_test.shape[0], 28, 28,1)
y_train = y.reshape(-1,1)

__Splitting Data__

In [10]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size = 0.20, 
                                                      random_state = 141)

__Clear the Backend__

In [11]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

__Piecewise Constant Scheduling__
* Added an additional learning constant
* A dynamic learning process

In [12]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 10:
        return 0.005
    elif epoch < 15:
        return 0.003
    elif epoch < 20:
        return 0.001
    else:
        return 0.0001

In [13]:
def piecewise_constant(boundaries, values):
    boundaries = np.array([0] + boundaries)
    values = np.array(values)
    def piecewise_constant_fn(epoch):
        return values[np.argmax(boundaries > epoch) - 1]
    return piecewise_constant_fn
piecewise_constant_fn = piecewise_constant([5,10,15,20,25], [0.01, 0.005,0.003,0.001,0.0001])

In [14]:
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

__Model Creation__

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=64, kernel_size=(5, 5),padding="valid" ,
                              activation='elu',kernel_initializer="lecun_normal", input_shape=(28,28,1)))
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters=32, kernel_size=(5, 5),padding="valid", 
                              activation='elu',kernel_initializer="lecun_normal"))
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.GaussianDropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(units=128, activation='selu',kernel_initializer="lecun_normal"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(units=64, activation='selu',kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(units=10, activation = 'softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        1664      
                                                                 
 average_pooling2d (AverageP  (None, 12, 12, 64)       0         
 ooling2D)                                                       
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 32)          51232     
                                                                 
 average_pooling2d_1 (Averag  (None, 4, 4, 32)         0         
 ePooling2D)                                                     
                                                        

__Model Compile__

In [17]:
model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.Adam(clipvalue=1.0,clipnorm=1.0),metrics=["accuracy"])

__Model Fitting__

In [18]:
history = model.fit(x_train,y_train,epochs=25,validation_data=(x_valid,y_valid),callbacks=[lr_scheduler])

Epoch 1/25
1050/1050 [==============================] - 13s 5ms/step - loss: 0.2471 - accuracy: 0.9267 - val_loss: 0.1159 - val_accuracy: 0.9674 - lr: 0.0100
Epoch 2/25
1050/1050 [==============================] - 5s 5ms/step - loss: 0.1222 - accuracy: 0.9643 - val_loss: 0.1094 - val_accuracy: 0.9687 - lr: 0.0100
Epoch 3/25
1050/1050 [==============================] - 5s 5ms/step - loss: 0.1019 - accuracy: 0.9701 - val_loss: 0.0872 - val_accuracy: 0.9763 - lr: 0.0100
Epoch 4/25
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0908 - accuracy: 0.9741 - val_loss: 0.0541 - val_accuracy: 0.9855 - lr: 0.0100
Epoch 5/25
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0862 - accuracy: 0.9763 - val_loss: 0.0705 - val_accuracy: 0.9826 - lr: 0.0100
Epoch 6/25
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0448 - accuracy: 0.9862 - val_loss: 0.0444 - val_accuracy: 0.9883 - lr: 0.0050
Epoch 7/25
1050/1050 [==============================] - 5

__Save the Model__
* I already saved this model to local and cloud

In [19]:
#model.save("Digit_Recog_Mod_2.h5")

In [20]:
#ml = keras.models.load_model("Digit_Recog_Mod_2.h5")

__Testing Model Against Test Data__

In [21]:
pred = model.predict(test_df)

__Get the index of the max value in the array__

In [22]:
prediction = [np.argmax(i) for i in pred]

In [23]:
img_id = list(range(1,28001))
img_id = np.array(img_id)

In [24]:
img_id.shape

(28000,)

__Final Dataframe of Predictions__
* Has to be in this format for the competition

In [25]:
df = pd.DataFrame({"ImageId":img_id,"Label":prediction})
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [26]:
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [27]:
df.tail()

,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2


__Download to csv format__

In [28]:
#df.to_csv("Michael_Woo_Predictions_Digit_Recog_2.csv",index=False)